In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import Error
pd.options.display.float_format = '{:,}'.format

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
DIR = '/content/gdrive/My Drive/Goodreads/data'
DIR_GENRE = '/content/gdrive/My Drive/Goodreads/data/genre/'

In [0]:
df_reviews = pd.read_csv(os.path.join(DIR, 'df_reviews_short.csv'))

In [6]:
print(df_reviews.shape)
df_reviews.head()

(55017, 7)


,book_id,review_text,average_rating,image_url,title,description,ratings_count
0,2767052,I cracked and finally picked this up. Very enj...,4.34,https://images.gr-assets.com/books/1447303603m...,"The Hunger Games (The Hunger Games, #1)",Winning will make you famous.\nLosing means ce...,4899965
1,3,Tuve el gusto de leerlo antes de que saliera l...,4.45,https://images.gr-assets.com/books/1474154022m...,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter's life is miserable. His parents ...,4765497
2,41865,"If I was 15, I would have LOVED this. It wasn'...",3.57,https://images.gr-assets.com/books/1361039443m...,"Twilight (Twilight, #1)",About three things I was absolutely positive.\...,3941381
3,2657,"Still relevant and beautiful., R.I.P. Ms. Harp...",4.26,https://images.gr-assets.com/books/1361975680m...,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,3255518
4,4671,I read this classic twice in high school. Its ...,3.89,https://images.gr-assets.com/books/1490528560m...,The Great Gatsby,"THE GREAT GATSBY, F. Scott Fitzgerald's third ...",2758812


In [0]:
df_reviews = df_reviews.head(500)

In [0]:
# return the wordnet object value corresponding to the POS tag
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer


contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", \
                    "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", \
                    "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", \
                    "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", \
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)


def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    text = replace_contractions(text)
    return(text)

contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", \
                    "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", \
                    "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", \
                    "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", \
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

In [9]:
df_reviews["review_clean"] = 1
print(len(df_reviews))

500


In [10]:
for i in range (len(df_reviews)):
    df_reviews["review_clean"][i] = clean_text(df_reviews["review_text"][i])
    if (i % 10 == 0 or i <= 30):
        print(i, end=',\n')
    
#df_reviews["review_clean"] = df_reviews["review_text"].apply(lambda x: clean_text(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0,
1,
2,
3,
4,
5,
6,
7,
8,
9,
10,
11,
12,
13,
14,
15,
16,
17,
18,
19,
20,
21,
22,
23,
24,
25,
26,
27,
28,
29,
30,
40,
50,
60,
70,
80,
90,
100,
110,
120,
130,
140,
150,
160,
170,
180,
190,
200,
210,
220,
230,
240,
250,
260,
270,
280,
290,
300,
310,
320,
330,
340,
350,
360,
370,
380,
390,
400,
410,
420,
430,
440,
450,
460,
470,
480,
490,


In [0]:
export_csv = df_reviews.to_csv (r'/content/gdrive/My Drive/Goodreads/data/df_reviews500.csv', index = None, header=True)

In [13]:
df_reviews.head()

,book_id,review_text,average_rating,image_url,title,description,ratings_count,review_clean
0,2767052,I cracked and finally picked this up. Very enj...,4.34,https://images.gr-assets.com/books/1447303603m...,"The Hunger Games (The Hunger Games, #1)",Winning will make you famous.\nLosing means ce...,4899965,crack finally pick enjoyable quick read put li...
1,3,Tuve el gusto de leerlo antes de que saliera l...,4.45,https://images.gr-assets.com/books/1474154022m...,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter's life is miserable. His parents ...,4765497,tuve el gusto de leerlo ante de que saliera la...
2,41865,"If I was 15, I would have LOVED this. It wasn'...",3.57,https://images.gr-assets.com/books/1361039443m...,"Twilight (Twilight, #1)",About three things I was absolutely positive.\...,3941381,would love terrible fact perfect quick read ra...
3,2657,"Still relevant and beautiful., R.I.P. Ms. Harp...",4.26,https://images.gr-assets.com/books/1361975680m...,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,3255518,still relevant beautiful r.i.p harper lee feb ...
4,4671,I read this classic twice in high school. Its ...,3.89,https://images.gr-assets.com/books/1490528560m...,The Great Gatsby,"THE GREAT GATSBY, F. Scott Fitzgerald's third ...",2758812,read classic twice high school hard like book ...


In [14]:
df_reviews["review_clean"][1]

'tuve el gusto de leerlo ante de que saliera la pelicula xd remember try time read always give page skip book read whole series come back book hard review book without take series whole reading series make feel like child excite first book watch movie read book happy face character know make book much good actual visual book much movie movie good best harry potter book i am late hp frenzy cannot tell review reflection fatigue actual opinion fine fun uplift change may expect much young adult book read book like okay compare hp sure value they are entertain one imprint soul keep perservering series kind self-respecting english teacher would finish g-damn series i am tell book rock change look forward day re-read august begin make deal see book come way exchange three adult book make interesting probably attempt begin give proper series critique finish probably though know i would assume know lazy h*ck really think one pertains troll release harry ron deal intelligence face threat physica

In [15]:
df_reviews.head()

,book_id,review_text,average_rating,image_url,title,description,ratings_count,review_clean
0,2767052,I cracked and finally picked this up. Very enj...,4.34,https://images.gr-assets.com/books/1447303603m...,"The Hunger Games (The Hunger Games, #1)",Winning will make you famous.\nLosing means ce...,4899965,crack finally pick enjoyable quick read put li...
1,3,Tuve el gusto de leerlo antes de que saliera l...,4.45,https://images.gr-assets.com/books/1474154022m...,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter's life is miserable. His parents ...,4765497,tuve el gusto de leerlo ante de que saliera la...
2,41865,"If I was 15, I would have LOVED this. It wasn'...",3.57,https://images.gr-assets.com/books/1361039443m...,"Twilight (Twilight, #1)",About three things I was absolutely positive.\...,3941381,would love terrible fact perfect quick read ra...
3,2657,"Still relevant and beautiful., R.I.P. Ms. Harp...",4.26,https://images.gr-assets.com/books/1361975680m...,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,3255518,still relevant beautiful r.i.p harper lee feb ...
4,4671,I read this classic twice in high school. Its ...,3.89,https://images.gr-assets.com/books/1490528560m...,The Great Gatsby,"THE GREAT GATSBY, F. Scott Fitzgerald's third ...",2758812,read classic twice high school hard like book ...


In [16]:
#Fit TFIDF 
#Learn vocabulary and tfidf from all style_ids.

import pickle
#from nltk.corpus import stopwords
#from nltk.stem import SnowballStemmer
#from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.metrics.pairwise import cosine_similarity
#from scipy import sparse
#import re
#import gensim
#from skimage import io
#import matplotlib.pyplot as plt
#from gensim.test.utils import get_tmpfile

tf = TfidfVectorizer(analyzer='word', 
                     min_df=10,
                     ngram_range=(1, 2),
                     #max_features=1000,
                     stop_words='english')
tf.fit(df_reviews['review_clean'])

#Transform style_id products to document-term matrix.
tfidf_matrix = tf.transform(df_reviews['review_clean'])
#pickle.dump(tf, open("../models/tfidf_model.pkl", "wb"))


    
path_model = '/content/gdrive/My Drive/Goodreads/models/' 
os.makedirs(path_model, exist_ok=True) 
pickle.dump(tf, open(path_model+'tfidf_model.pkl', "wb"))


print(tfidf_matrix.shape)

(500, 508761)


In [17]:
# Compress with SVD
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=500)
latent_matrix = svd.fit_transform(tfidf_matrix)
pickle.dump(svd, open(path_model+"svd_model.pkl", "wb"))

print(latent_matrix.shape)

(500, 500)


In [18]:
n = 25 #pick components
#Use elbow and cumulative plot to pick number of components. 
#Need high ammount of variance explained. 
doc_labels = df_reviews[['book_id','title','image_url']]
svd_feature_matrix = pd.DataFrame(latent_matrix[:,0:n] ,index=doc_labels)
print(svd_feature_matrix.shape)
svd_feature_matrix.head()

pickle.dump(svd_feature_matrix, open(path_model+"lsa_embeddings.pkl", "wb"))

(500, 25)


Doc2Vec Model



In [0]:
import re
import gensim

#Use reviews for vocabulary 
reviews = df_reviews.review_clean.values.tolist()

#print(reviews[0])

documents = []
for i in range(len(df_reviews)):
    mystr = reviews[i]
    #mystr = mystr + descriptions[i]
    documents.append(re.sub("[^\w]", " ",  mystr).split())

In [20]:
print(len(df_reviews))
print(len(documents))

500
500


In [0]:
import re
import gensim

formatted_documents = [gensim.models.doc2vec.TaggedDocument(doc, [i]) for i, doc in enumerate(documents)]

model = gensim.models.doc2vec.Doc2Vec(vector_size=25, min_count=5, epochs=200, seed=0, window=3, dm=1)
model.build_vocab(formatted_documents)

In [22]:
len(formatted_documents)

500

In [23]:
%time model.train(formatted_documents, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 47min 58s, sys: 8.29 s, total: 48min 6s
Wall time: 17min 3s


In [24]:
from gensim.test.utils import get_tmpfile

fname = get_tmpfile(path_model+"doc2vec_model")
model.save(path_model+"doc2vec_model")
model = gensim.models.doc2vec.Doc2Vec.load(path_model+"doc2vec_model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [25]:
vector = model.infer_vector(doc_words=["this", "is", "a", "test"], epochs=50)
vector

array([ 0.29929236, -0.60735446,  0.46196327,  0.12014095, -0.709318  ,
        0.4498157 , -0.29343975, -0.18569705,  0.7595433 , -0.36876082,
        0.5192237 , -0.18593471,  1.0403113 ,  0.30184215, -0.22479591,
       -0.33051562,  0.25337845,  0.30646053,  0.32459798,  0.63471866,
        0.10305256, -0.5533143 , -0.8721733 ,  0.62372166,  0.22955567],
      dtype=float32)

In [26]:
doctovec_feature_matrix = pd.DataFrame(model.docvecs.vectors_docs, index=doc_labels)
print(doctovec_feature_matrix.shape)
doctovec_feature_matrix.head(3)
pickle.dump(doctovec_feature_matrix, open(path_model+"doctovec_embeddings.pkl", "wb"))

(500, 25)


In [27]:
doc_labels

,book_id,title,image_url
0,2767052,"The Hunger Games (The Hunger Games, #1)",https://images.gr-assets.com/books/1447303603m...
1,3,Harry Potter and the Sorcerer's Stone (Harry P...,https://images.gr-assets.com/books/1474154022m...
2,41865,"Twilight (Twilight, #1)",https://images.gr-assets.com/books/1361039443m...
3,2657,To Kill a Mockingbird,https://images.gr-assets.com/books/1361975680m...
4,4671,The Great Gatsby,https://images.gr-assets.com/books/1490528560m...
5,11870085,The Fault in Our Stars,https://images.gr-assets.com/books/1360206420m...
6,5907,The Hobbit,https://images.gr-assets.com/books/1372847500m...
7,5107,The Catcher in the Rye,https://images.gr-assets.com/books/1398034300m...
8,1885,Pride and Prejudice,https://images.gr-assets.com/books/1320399351m...
9,960,"Angels & Demons (Robert Langdon, #1)",https://images.gr-assets.com/books/1303390735m...
